# nuImages devkit tutorial

Welcome to the nuImages tutorial.
This demo assumes the database itself is available at `/data/sets/nuimages`.

## A Gentle Introduction to nuImages

In this part of the tutorial, let us go through a top-down introduction of our database. Our dataset is structured as a relational database with tables, tokens and foreign keys. The tables are the following:

1. `log` - Log from which the sample was extracted.
2. `sample` - An annotated camera image with an associated timestamp and past and future images and pointclouds.
3. `sample_data` - An image or pointcloud associated with a sample.
4. `ego_pose` - The vehicle ego pose and timestamp associated with a sample_data.
5. `sensor` - General information about a sensor, e.g. `CAM_BACK_LEFT`.
6. `calibrated_sensor` - Calibration information of a sensor in a log.
7. `category` - Taxonomy of object and surface categories (e.g. `vehicle.car`, `flat.drivable_surface`). 
8. `attribute` - Property of an object that can change while the category remains the same.
9. `object_ann` - Bounding box and mask annotation of an object (e.g. car, adult).
10. `surface_ann` - Mask annotation of a surface (e.g. `flat.driveable surface` and `vehicle.ego`).

## nuImages Basics


### Initialization
To initialize the dataset class, we run the following:

In [ ]:
%matplotlib inline
from nuimages import NuImages

nuim = NuImages(dataroot='/data/sets/nuimages', version='v1.0-test', verbose=True, lazy=True)

We can change the dataroot parameter if the dataset is installed in a different folder. We can also emit it to use the default setup.

### Tables

As described above, the NuImages class holds several tables. Each table is a list of records, and each record is a dictionary. For example the first record of the category table is stored at:

In [ ]:
nuim.category[0]

To see the list of all tables, simply refer to the `table_names` variable:

In [ ]:
nuim.table_names

### Lazy loading

Initializing the NuImages instance above was very fast, as we did not actually load the tables. Rather, the class implements lazy loading that overwrites the internal `__getattr__()` function to load a table if it is not already stored in memory. The moment we accessed `category`, we could see the table being loaded from disk. To disable such notifications, just set `verbose=False` when initializing the NuImages object. Furthermore lazy loading can be disabled with `lazy=False`.

### Indexing

Since all tables are lists of dictionaries, we can use standard Python operations on them. A very common operation is to retrieve a particular record by its token. Since this operation takes linear time, we precompute an index that helps to access a record in constant time.

Let us select the first image in this dataset version and split:

In [ ]:
sample = nuim.sample[0]
sample

We can also get the sample record from a sample token:

In [ ]:
sample = nuim.get('sample', sample['token'])
sample

What this does is actually to lookup the index:

In [ ]:
sample_idx = nuim.getind('sample', sample['token'])

### Rendering

To render an annotated image we use the `render_sample()` function. We can see the boxes and masks for each object category, as well as the surfaces masks for ego vehicle and driveable surface. At the top left corner of each box, we see the name of the object category. We use the following colors:
- vehicles: orange
- bicycles and motorcycles: red
- pedestrians: blue
- cones and barriers: black
- driveable surface: teal / green

In [ ]:
im = nuim.render_sample(nuim.sample[0]['token'])

### Statistics

The `list_*()` methods are useful to get an overview of the dataset dimensions. Note that these statistics are always *for the current split* that we initialized the `NuImages` instance with, rather than the entire dataset.

In [ ]:
nuim.list_logs()

`list_categories()` lists the category frequencies, as well as the category name and description. Each category is either an object or a surface, but not both.

In [ ]:
nuim.list_categories()

We can also specify a `sample_tokens` parameter for `list_categories()` to get the category statistics for a particular set of samples.

In [ ]:
sample_tokens = [nuim.sample[9]['token']]
nuim.list_categories(sample_tokens=sample_tokens)

`list_attributes()` shows the frequency, name and description of all attributes:

In [ ]:
nuim.list_attributes()

`list_cameras()` shows us how many camera entries and samples there are for each channel, such as the front camera.
Each camera uses slightly different intrinsic parameters, which will be provided in a future release.

In [ ]:
nuim.list_cameras()

`list_sample_content()` shows for each sample all the associated sample_datas, which are images (from the same camera) and lidar pointclouds. There can be up to 13 images and 13 pointclouds, spaced evenly at 500ms +- 250ms. However, a small percentage of the samples has less sample_datas, either because they were at the beginning or end of a log, or due to delays or dropped data packages.

In [ ]:
nuim.list_sample_content(nuim.sample[0]['token'])